In [2]:
import os
import pandas as pd
import numpy as np
import pickle
import tensorflow as tf
from matplotlib.image import imread
from numpy import mean
from numpy import std
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.layers import Dense, Dropout, Flatten
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import *
print(f"Tensorflow version: {tf.__version__}")
print(f"Keras Version: {tf.keras.__version__}")

print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

Tensorflow version: 2.3.0
Keras Version: 2.4.0
GPU is NOT AVAILABLE


In [3]:
data_dev = []
data_test = []
data_train = []
file_name_dev = "audioDataDev.pickle"
file_name_test = "audioDataTest.pickle"
file_name_train = "audioDataTrain.pickle"

with (open(file_name_dev, "rb")) as f:
    while True:
        try:
            data_dev.append(pickle.load(f))
        except EOFError:
            break
f.close()

with (open(file_name_test, "rb")) as f:
    while True:
        try:
            data_test.append(pickle.load(f))
        except EOFError:
            break
f.close()

with (open(file_name_train, "rb")) as f:
    while True:
        try:
            data_train.append(pickle.load(f))
        except EOFError:
            break
f.close()

In [4]:
X_dev = []
y_dev = []
X_train = []
y_train = []
X_test = []
y_test = []

for i in range(len(data_dev)-1):
    X_dev.append(data_dev[i+1][3])
for i in range(len(data_dev)-1):
    y_dev.append(data_dev[i+1][5:12])
    
for i in range(len(data_test)-1):
    X_test.append(data_test[i+1][3])
for i in range(len(data_test)-1):
    y_test.append(data_test[i+1][5:12])
    
for i in range(len(data_train)-1):
    X_train.append(data_train[i+1][3])
for i in range(len(data_train)-1):
    y_train.append(data_train[i+1][5:12])

X_dev = np.array(X_dev)
y_dev = np.array(y_dev)
X_test = np.array(X_test)
y_test = np.array(y_test)
X_train = np.array(X_train)
y_train = np.array(y_train)


In [5]:
y_dev = y_dev * 3
y_test = y_test * 3
y_train = y_train * 3
y_dev = y_dev.round()
y_test = y_test.round()
y_train = y_train.round()

In [6]:
X_train.shape

(16327, 128, 850)

In [12]:
# get the model
def get_model():
    model = Sequential()
    model.add(Conv1D(32, kernel_size=(2), activation='relu', input_shape=(128, 850)))
    model.add(MaxPooling1D(pool_size=(2)))
    model.add(Dropout(0.25))
    model.add(Conv1D(filters=64, kernel_size=(5), activation='relu'))
    model.add(MaxPooling1D(pool_size=(2)))
    model.add(Dropout(0.25))
    model.add(Conv1D(filters=128, kernel_size=(5), activation="relu"))
    model.add(MaxPooling1D(pool_size=(2)))
    model.add(Dropout(0.25))
    model.add(Conv1D(filters=256, kernel_size=(5), activation="relu"))
    model.add(MaxPooling1D(pool_size=(2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='sigmoid'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='sigmoid'))
    model.add(Dropout(0.5))
    model.add(Dense(7, activation='sigmoid'))
    return model
 
# evaluate a model using repeated k-fold cross-validation
adam = Adam(0.0005)
model = get_model()
model.compile(optimizer=adam,loss='mean_squared_error',metrics=['MeanSquaredError','AUC','accuracy'])
model.fit(X_train, y_train, epochs=10, validation_data=(X_dev, y_dev), batch_size=1000)
yhat = model.predict(X_test)

Epoch 1/20
17/17 [==============================] - 10s 610ms/step - loss: 2.8241 - mean_squared_error: 2.8241 - auc: 0.5515 - accuracy: 0.1738 - val_loss: 2.2736 - val_mean_squared_error: 2.2736 - val_auc: 0.6719 - val_accuracy: 0.2117
Epoch 2/20
17/17 [==============================] - 9s 529ms/step - loss: 2.7496 - mean_squared_error: 2.7496 - auc: 0.6177 - accuracy: 0.2368 - val_loss: 2.2513 - val_mean_squared_error: 2.2513 - val_auc: 0.6632 - val_accuracy: 0.2117
Epoch 3/20
17/17 [==============================] - 9s 510ms/step - loss: 2.7337 - mean_squared_error: 2.7337 - auc: 0.6484 - accuracy: 0.2370 - val_loss: 2.2439 - val_mean_squared_error: 2.2439 - val_auc: 0.6756 - val_accuracy: 0.2117
Epoch 4/20
17/17 [==============================] - 9s 504ms/step - loss: 2.7223 - mean_squared_error: 2.7223 - auc: 0.6697 - accuracy: 0.2392 - val_loss: 2.2402 - val_mean_squared_error: 2.2402 - val_auc: 0.7033 - val_accuracy: 0.2117
Epoch 5/20
17/17 [==============================] - 9s 

In [13]:
print(yhat[3000])
print(y_test[1].shape)
print(y_test[3000])

[0.01108488 0.98947203 0.55688107 0.70674175 0.11051086 0.6799463
 0.05487254]
(7,)
[2. 3. 0. 0. 0. 0. 0.]


In [9]:
results=[]
total = 0
similar = 0
for i in range(len(y_test)):
    for j in range(len(y_test[i])):
        if y_test[i][j] == yhat[i][j]:
            similar += 1
        total += 1
acc = similar/total

print('>%.3f' % acc)
results.append(acc)

>0.000
